### Load Dataset

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Dev Process

Assume the task we want to classify is determining the label $y$ of observed data $X=(x_1, x_2)$, where $x_1, x_2$ are the attributes of $X$. Further, we assume attribute $x_1 \in A_1 = \{1, 2, 3\}$ and attribute $x_2 \in A_2 = \{S, M, L\}$. And the label $y \in C = \{1, -1\}$. The observed data and labels are below.

The assumed problem is to predict the data $X = \{2, S\}^T$, which is supposed to get estimation value $-1$ based on the Multinomial Naive Bayes.

In [2]:
import gen_synthetic as gs

# Test the randomly generated data
x1_dom = [1, 2, 3]
x2_dom = ['S', 'M', 'L']
x_dom = [x1_dom, x2_dom]
y_dom = [-1, 1]

data_random = gs.gen_naive_bayes_synthetic(x_dom, y_dom, 15)
print 'data_random: \n', data_random

# Test embedded hard code data
data_hard_code = gs.gen_naive_bayes_synthetic([], [], 0, hard_code=True)
print 'data_hard_code: \n', data_hard_code

data_random: 
{'data': [[3, 'M'], [2, 'M'], [1, 'S'], [2, 'S'], [1, 'L'], [2, 'S'], [2, 'S'], [2, 'M'], [3, 'L'], [1, 'M'], [3, 'M'], [3, 'M'], [3, 'L'], [2, 'L'], [1, 'S']], 'target': [-1, 1, 1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1]}
data_hard_code: 
{'data': [[1, 'S'], [1, 'M'], [1, 'M'], [1, 'S'], [1, 'S'], [2, 'S'], [2, 'M'], [2, 'M'], [2, 'L'], [2, 'L'], [3, 'L'], [3, 'M'], [3, 'M'], [3, 'L'], [3, 'L']], 'target': [-1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1]}


In [3]:
import pandas as pd
tt = np.array([[1, 1], [2, 2]])
df = pd.DataFrame(tt)
ty = np.array([10, 20])
df[3] = ty
print df

   0  1   3
0  1  1  10
1  2  2  20


### Naive Bayes without Laplace smoothing

In [4]:
from classifiers import MultinomialNB
mnNB = MultinomialNB()

mnNB.train(np.array(data_random['data']), np.array(data_random['target']))
df = mnNB.counts_table
print df[df[0] == '1']

    0  1  2
2   1  S  1
4   1  L  1
9   1  M -1
14  1  S -1


In [5]:
mnNB.train(np.array(data_hard_code['data']), np.array(data_hard_code['target']))
df = mnNB.counts_table
print df
print df[df[0] == '1']
print 'number of above rows: ', len(df[df[0] == '1'])

print df[(df[0] == '1') & (df[1] == 'M')]
print 'number of above rows: ', len(df[(df[0] == '1') & (df[1] == 'M')])

print 'number of non-existing rows:', len(df[df[0] == '7'])

print '\nThe probability table is:'
for k in sorted(mnNB.prob_table):
    print 'k = %s, prob = %f' % (k, mnNB.prob_table[k])

    0  1  2
0   1  S -1
1   1  M -1
2   1  M  1
3   1  S  1
4   1  S -1
5   2  S -1
6   2  M -1
7   2  M  1
8   2  L  1
9   2  L  1
10  3  L  1
11  3  M  1
12  3  M  1
13  3  L  1
14  3  L -1
   0  1  2
0  1  S -1
1  1  M -1
2  1  M  1
3  1  S  1
4  1  S -1
number of above rows:  5
   0  1  2
1  1  M -1
2  1  M  1
number of above rows:  2
number of non-existing rows: 0

The probability table is:
k = -1, prob = 0.400000
k = 1, prob = 0.600000
k = 1|-1, prob = 0.222222
k = 1|1, prob = 0.333333
k = 2|-1, prob = 0.333333
k = 2|1, prob = 0.500000
k = 3|-1, prob = 0.444444
k = 3|1, prob = 0.166667
k = L|-1, prob = 0.222222
k = L|1, prob = 0.333333
k = M|-1, prob = 0.444444
k = M|1, prob = 0.333333
k = S|-1, prob = 0.333333
k = S|1, prob = 0.333333


### Laplace smoothing

In [6]:
mnNB.train(np.array(data_hard_code['data']), np.array(data_hard_code['target']), laplaceS=1.0)
df = mnNB.counts_table
print df[df[0] == '1']
print 'number of above rows: ', len(df[df[0] == '1'])

print df[(df[0] == '1') & (df[1] == 'M')]
print 'number of above rows: ', len(df[df[0] == '1'])

print 'number of non-existing rows:', len(df[df[0] == '7'])

print '\nThe probability table is:'
for k in sorted(mnNB.prob_table):
    print 'k = %s, prob = %f' % (k, mnNB.prob_table[k])

   0  1  2
0  1  S -1
1  1  M -1
2  1  M  1
3  1  S  1
4  1  S -1
number of above rows:  5
   0  1  2
1  1  M -1
2  1  M  1
number of above rows:  5
number of non-existing rows: 0

The probability table is:
k = -1, prob = 0.400000
k = 1, prob = 0.600000
k = 1|-1, prob = 0.222222
k = 1|1, prob = 0.333333
k = 2|-1, prob = 0.333333
k = 2|1, prob = 0.500000
k = 3|-1, prob = 0.444444
k = 3|1, prob = 0.166667
k = L|-1, prob = 0.222222
k = L|1, prob = 0.333333
k = M|-1, prob = 0.444444
k = M|1, prob = 0.333333
k = S|-1, prob = 0.333333
k = S|1, prob = 0.333333


In [7]:
XX_test = [2, 'S']
mnNB.predict(np.array(XX_test))

-1


# Assignment

As part of this problem, you will implement a Naive Bayes classifier for classifying movie reviews as positive or negative. The dataset that you will be using is the IMDB Large Movie Review dataset (Maas et. al, ACL 2011). The processed dataset can be found [here](https://www.dropbox.com/s/liz0o40f5mpj8ye/hw1_dataset_nb.tar.gz?dl=0). The task is to estimate appropriate parameters using the training data, and use it to predict reviews from the test data, and classify each of them as either positive or negative.

We employ the *Multinomial Naive Bayes model for modeling each $P(X_i | Y = y_k)$ ($i = 1 .. n$), with appropriate word counts* (Note $n$ is the number of dimensions).

Please use Matlab, Python, R, C/C++ or Java for your implementation. Note that you will have to submit your codes in Autolab, and provide the answers to the questions in the below subsections in your report.

### Preprocessing
The dataset is partitioned into 2 folders: `train` and `test`, each of which contains 2 subfolders (`pos` and `neg`, for positive and negative samples respectively). The content of each file has to be converted to a bag-of-words representation. So the first task is to go through all the files in the `train` folder, and construct the vocabulary $V$ of all unique words. Please ignore all the stop-words as given in the file `sw.txt` (provided along with the dataset). The words from each file (both in training and testing phase) must be extracted by splitting the raw text only with whitespace characters and {\color{red}converting them to lowercase characters}. 

The next step is to get counts of each individual words for the positive and the negative classes separately, to get $P(word | class)$. 

### Classification
In this step, you need to go through all the negative and positive samples in the test data, and classify each sample according to the parameters learned earlier. The classification should be done by comparing the log-posterior (un-normalized), which is given by $\log(P(X|Y)P(Y))$, for both the classes.

### Laplace smoothing
An issue with the original Naive Bayes setup is that if a test sample contains a word which is not present in the dictionary, the $P(word|label)$ goes to $0$. To mitigate this issue, one solution is to employ Laplace smoothing (it has a parameter $\alpha$). Augment your $P(word | class)$ calculations by including the appropriate terms for doing Laplace smoothing. 

Report the confusion matrix and overall accuracy of your classifier on the test dataset with $\alpha = 1$. Recall that the confusion matrix for such 2-class classification problem, is a matrix of the number of true positives (positive samples correctly classified as positive), number of true negatives (negative samples correctly classified as negative), number of false positives (negative samples incorrectly classified as positive), and number of false negatives (positive samples incorrectly classified as negative). The accuracy is the ratio of sum of true positives and true negatives, and the total number of samples (in the test dataset).

Now vary the value of $\alpha$ from $0.0001$ to $1000$ (by multiplying $\alpha$ with 10 each time), and report a plot of the accuracy on the test dataset for the corresponding values of $\alpha$. (The x-axis should represent $\alpha$ values and use a $\log$ scale for the x-axis).

 Why do you think the accuracy suffers when $\alpha$ is too high or too low? 

In [8]:
import numpy as np
import pandas as pd
import os

fileName = '0_3.txt'
dataPath = os.path.join('dataset', 'hw1_dataset_nb', 'train', 'neg', fileName)
data = open(dataPath, 'r').read()
print data

Story of a man who has unnatural feelings for a pig  Starts out with a opening scene that is a terrific example of absurd comedy  A formal orchestra audience is turned into an insane  violent mob by the crazy chantings of it's singers  Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting  Even those from the era should be turned off  The cryptic dialogue would make Shakespeare seem easy to a third grader  On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond  Future stars Sally Kirkland and Frederic Forrest can be seen briefly 


### Read training, testing data and stop words

In [9]:
# Get IMDB data
def getIMDBData(dirPre, dataLabel):
    res = {}
    for dl in dataLabel:
        dirPath = os.path.join(dirPre, dl)
        fileNames = os.listdir(dirPath)

        docs = []
        for fN in fileNames:
            doc = open(os.path.join(dirPath, fN), 'r').read()
            docs.append(doc)
            
        res[dl] = docs
        
    return res
    
dataPathPre = os.path.join('dataset', 'hw1_dataset_nb')
devType = ['train', 'test']
dataLabel = ['pos', 'neg']

IMDB_train = getIMDBData(os.path.join(dataPathPre, devType[0]), dataLabel)
IMDB_test = getIMDBData(os.path.join(dataPathPre, devType[1]), dataLabel)
IMDB_stop_words = open(os.path.join(dataPathPre, 'sw.txt'), 'r').read()

In [10]:
# Convert text data to words list
from dataset import removePunctuation

def doc2WordList(X, dataLabel, sw):
    """
    Input
    -----
    - X: dict contains 'pos' and 'neg', which is defined in dataLabel.
         Each X[dataLabel[0]] contains docs list, i.e. [doc1, doc2, ..., docN]
    
    Remove punctuations, and make words in lowercase.
    """
    res_X = {}
    
    for dl in dataLabel:
        res_X[dl] = []
        for doc in X[dl]:
            res_X[dl].append([x for x in removePunctuation(doc).split() if (not x in sw) and (len(x) > 0)])
            
    return res_X

stop_words = IMDB_stop_words.split()
X_train_list = doc2WordList(IMDB_train, dataLabel, stop_words)
X_test_list = doc2WordList(IMDB_test, dataLabel, stop_words)

In [11]:
print stop_words[0:5]

['a', 'about', 'above', 'across', 'after']


In [12]:
print X_train_list['neg'][0]

['story', 'unnatural', 'feelings', 'pig', 'starts', 'scene', 'terrific', 'example', 'absurd', 'comedy', 'formal', 'orchestra', 'audience', 'insane', 'violent', 'mob', 'crazy', 'chantings', 'singers', 'unfortunately', 'stays', 'absurd', 'time', 'narrative', 'eventually', 'putting', 'era', 'cryptic', 'dialogue', 'shakespeare', 'easy', 'third', 'grader', 'technical', 'level', 'cinematography', 'future', 'vilmos', 'zsigmond', 'future', 'stars', 'sally', 'kirkland', 'frederic', 'forrest', 'seen', 'briefly']


In [13]:
print '****** training data: ******'
for i, tx in enumerate(X_train_list['neg'][0:5]):
    print '%d -----------' % i
    print tx
       
print '\n neg ================> pos \n'

for i, tx in enumerate(X_train_list['pos'][0:5]):
    print '%d -----------' % i
    print tx

print '\n------------------------------------------\n'

print '****** testing data: ******'    
for i, tx in enumerate(X_train_list['neg'][0:5]):
    print '%d -----------' % i
    print tx
       
print '\n neg ================> pos \n'

for i, tx in enumerate(X_train_list['pos'][0:5]):
    print '%d -----------' % i
    print tx


****** training data: ******
0 -----------
['story', 'unnatural', 'feelings', 'pig', 'starts', 'scene', 'terrific', 'example', 'absurd', 'comedy', 'formal', 'orchestra', 'audience', 'insane', 'violent', 'mob', 'crazy', 'chantings', 'singers', 'unfortunately', 'stays', 'absurd', 'time', 'narrative', 'eventually', 'putting', 'era', 'cryptic', 'dialogue', 'shakespeare', 'easy', 'third', 'grader', 'technical', 'level', 'cinematography', 'future', 'vilmos', 'zsigmond', 'future', 'stars', 'sally', 'kirkland', 'frederic', 'forrest', 'seen', 'briefly']
1 -----------
['airport', '77', 'starts', 'brand', 'luxury', '747', 'plane', 'loaded', 'valuable', 'paintings', 'belonging', 'rich', 'businessman', 'philip', 'stevens', 'james', 'stewart', 'flying', 'bunch', 'vips', 'estate', 'preparation', 'public', 'museum', 'board', 'stevens', 'daughter', 'julie', 'kathleen', 'quinlan', 'son', 'luxury', 'jetliner', 'takes', 'planned', 'midair', 'plane', 'hijacked', 'copilot', 'chambers', 'robert', 'foxworth',

### Construct vocabulary list

In [14]:
vl = []
for d in ['neg', 'pos']:
    for doc in X_train_list[d]:
        for wd in doc:
            vl.append(wd)
        
voc_list = list(set(vl))

In [15]:
len(voc_list)

95073

In [16]:
voc_dict = {}
for i, k in enumerate(voc_list):
    voc_dict[k] = i

It's very inefficient to use list of list to store data in Python. The below example shows how big difference they're. We need to use matrix as much as possible to improve our computation efficiency.

In [17]:
(25000 /1024) * (95073 / 1024) * 4 / 1024

8

In [18]:
import array
c = array.array(str("i"))
c.append(0)
c.append(5)
print c

array('i', [0, 5])


In [19]:
# Convert text to Bag of Words representation
import scipy.sparse as sp
import array

def _make_int_array():
    """Construct an array.array of a type suitable for scipy.sparse indices."""
    return array.array(str("i"))

def frombuffer_empty(buf, dtype):
        if len(buf) == 0:
            return np.empty(0, dtype=dtype)
        else:
            return np.frombuffer(buf, dtype=dtype)
        
def doc2vec(doc_list, voc_list):
    res_vec = {} # [0] * len(voc_list)
    for word in doc_list:
        if word in voc_list:
            feature_indx = voc_dict[word]
            if feature_indx in res_vec:
                res_vec[feature_indx] += 1
            else:
                res_vec[feature_indx] = 1
        
    return res_vec

def dataVectorization_origin(X, dataLabel, voc_list):
    """
    dataBW = {}
    for dl in dataLabel:
        dataBW[dl] = []
        for doc in X[dl]:
            # dataBW[dl].append(doc2vec(doc, voc_list))
            for word in doc:
                if word in voc_list:
                    try:
                        feature_indx = voc_dict[word]
                        if feature_indx in dataBW:
                            dataBW[feature_indx] += 1
                        else:
                            dataBW[feature_indx] = 1
                    except KeyError:
                        # Ignore out-of-vocabulary items for fixed_vocab=True
                        continue
                        
                    
            
    return dataBW
    
    """    
    j_indices = []
    
    # get efficient array 'array.array'.
    indptr = _make_int_array()
    values = _make_int_array()
    
    # The first pointer should point to the 
    # initial location, i.e. position 0.
    #
    # It's used to construct the parameter of
    # 'csc_matrix((data, indices, indptr), [shape=(M, N)])'
    indptr.append(0)
    for dl in dataLabel:
        for doc in X[dl]:
            feature_counter = {}
            for feature in doc:
                try:
                    feature_idx = voc_dict[feature]
                    if not feature_idx in feature_counter:
                        feature_counter[feature_idx] = 1
                    else:
                        feature_counter[feature_idx] += 1
                except KeyError:
                    # Ignore out-of-vocabulary items for fixed_vocab=True
                    continue   
            
            # As array.array, it uses method 'extend'
            # instead of 'append' to append element.
            j_indices.extend(feature_counter.keys())
            values.extend(feature_counter.values())
            indptr.append(len(j_indices))
            
    
    # Turn Python 'list' into numpy 'np.array'
    j_indices = np.asarray(j_indices, dtype=np.intc)
    # Turn Python 'array.array' into numpy 'np.array'
    indptr = np.frombuffer(indptr, dtype=np.intc)
    # Almost the same as above, except 'len(values) == 0' situation
    values = frombuffer_empty(values, dtype=np.intc)

    dataBW = sp.csr_matrix((values, j_indices, indptr),
                      shape=(len(indptr) - 1, len(voc_dict)))

    dataBW.sort_indices()
    return dataBW

def dataVectorization(X, voc_list):
    num_pos = len(X['pos'])
    num_neg = len(X['neg'])
    
    # For X_train_list:
    #   - 25000 rows
    #   - 95073 columns
    #   - 4 bytes per element
    # In all: 25000 * 95073 * 4 / (1024 ** 4) bytes, i.e. 8GB
    # dataBW = sp.lil_matrix((num_pos + num_neg, len(voc_list)))
    dataBW = np.zeros((num_pos + num_neg, len(voc_list)))
    
    for x in xrange(num_pos):
        for doc in X['pos']:
            for word in doc:
                dataBW[x, voc_dict[word]] += 1
                
    for x in range(num_pos, num_pos + num_neg):
        for doc in X['neg']:
            for word in doc:
                dataBW[x, voc_dict[word]] += 1
            
    return dataBW

# print X_train['pos'][0]
# print doc2vec(X_train['pos'][0], voc_list)
# X_train = dataVectorization(X_train_list, dataLabel, voc_list)
# print X_train['pos'][0]
testData = {}
testData['pos'] = X_train_list['pos'][0:5]
testData['neg'] = X_train_list['neg'][0:5]
import time 

tic = time.time()
tt1 = dataVectorization_origin(testData, dataLabel, voc_list)
tc = time.time()
print 'List of list implementation time cost:', (tc - tic) / 6.0 


tic = time.time()
tt = dataVectorization(testData, voc_list)
tc = time.time()
print 'Matrix implementation time cost:', (tc - tic) / 6.0 
# print tt['neg'][0]

List of list implementation time cost: 0.000166654586792
Matrix implementation time cost: 0.000666658083598


In [20]:
totalTime = (len(X_train_list['pos']) + len(X_train_list['pos'])) * (tc - tic) / 6.0 
print 'Total time: %f min' % (totalTime / 60.0)

Total time: 0.277774 min


In [21]:
X_train1 = dataVectorization_origin(X_train_list, dataLabel, voc_list)
print X_train1

  (0, 1610)	1
  (0, 1949)	1
  (0, 2326)	1
  (0, 8106)	1
  (0, 10303)	1
  (0, 12014)	1
  (0, 15867)	1
  (0, 17926)	1
  (0, 18048)	1
  (0, 18891)	2
  (0, 20882)	4
  (0, 23473)	1
  (0, 26896)	1
  (0, 26988)	1
  (0, 27610)	4
  (0, 28216)	1
  (0, 31568)	1
  (0, 32558)	1
  (0, 33619)	1
  (0, 36011)	1
  (0, 37539)	1
  (0, 38610)	1
  (0, 39008)	1
  (0, 39337)	1
  (0, 44637)	1
  :	:
  (24999, 47393)	1
  (24999, 48149)	1
  (24999, 49671)	1
  (24999, 50355)	1
  (24999, 50561)	1
  (24999, 61113)	1
  (24999, 61671)	1
  (24999, 61793)	1
  (24999, 63596)	1
  (24999, 64951)	2
  (24999, 67912)	2
  (24999, 68345)	1
  (24999, 69756)	2
  (24999, 71047)	1
  (24999, 74385)	1
  (24999, 76088)	1
  (24999, 77075)	1
  (24999, 77493)	1
  (24999, 78160)	1
  (24999, 78936)	1
  (24999, 80838)	1
  (24999, 84845)	1
  (24999, 85588)	1
  (24999, 90814)	1
  (24999, 91428)	1


# Experiment with scikit-learn

In [22]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [23]:
for k in twenty_train:
    print k
    # print "(%s, %s)\n" % (k, twenty_train[k])

description
DESCR
filenames
target_names
data
target


In [24]:
print twenty_train.target[0:29]

[1 1 3 3 3 3 3 2 2 2 3 1 0 0 1 1 2 0 3 0 3 0 3 1 1 1 3 3 2]


In [25]:
len(twenty_train.data)

len(twenty_train.filenames)

2257

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
import time
tic = time.time()
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
print time.time() - tic
X_train_counts.shape

0.888000011444


(2257, 35788)

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

### Use IMDB data

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
import time
tic = time.time()
cvect = CountVectorizer(analyzer="word", stop_words=stop_words)
X_train = cvect.fit_transform(IMDB_train['pos'] + IMDB_train['neg'])
print time.time() - tic
X_train.shape

5.83599996567


(25000, 74195)

In [29]:
print X_train

  (0, 9176)	4
  (0, 10758)	1
  (0, 13383)	1
  (0, 52945)	1
  (0, 66361)	1
  (0, 51471)	1
  (0, 57539)	2
  (0, 38342)	1
  (0, 65201)	4
  (0, 641)	1
  (0, 65203)	1
  (0, 51412)	1
  (0, 37744)	1
  (0, 6653)	1
  (0, 57097)	1
  (0, 12827)	1
  (0, 53345)	1
  (0, 57727)	1
  (0, 64151)	1
  (0, 24430)	1
  (0, 33538)	1
  (0, 63241)	2
  (0, 48258)	1
  (0, 50295)	1
  (0, 49088)	1
  :	:
  (24999, 13660)	1
  (24999, 68226)	2
  (24999, 54182)	1
  (24999, 68818)	1
  (24999, 59866)	1
  (24999, 29859)	1
  (24999, 51389)	1
  (24999, 45532)	1
  (24999, 10118)	1
  (24999, 44749)	1
  (24999, 71639)	1
  (24999, 47619)	1
  (24999, 32316)	1
  (24999, 31216)	1
  (24999, 67135)	1
  (24999, 40241)	1
  (24999, 10369)	1
  (24999, 41998)	2
  (24999, 2124)	1
  (24999, 55466)	1
  (24999, 38780)	1
  (24999, 19727)	1
  (24999, 20261)	1
  (24999, 19731)	1
  (24999, 59182)	1


In [30]:
print len(([0] * len(IMDB_train['pos'])) + ([1] * len(IMDB_train['pos'])))

25000
